In [1]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP, POMDPPolicies
using POMDPToolbox
using Distributions
using Parameters
using BeliefUpdaters
using Plots
using StaticArrays

┌ Warning: Importing POMDPToolbox. As of Julia 0.7 POMDPToolbox has been discontinued. Please use POMDPModelTools, POMDPPolicies, BeliefUpdaters, and/or POMDPSimulators instead.
└ @ POMDPToolbox C:\Users\cveda\.julia\packages\POMDPToolbox\OdZy7\src\POMDPToolbox.jl:4
┌ Warning: Error requiring `UnicodePlots` from `Plots`
│   exception = (LoadError("C:\\Users\\cveda\\.julia\\packages\\Plots\\CBYO0\\src\\backends\\unicodeplots.jl", 77, UndefVarError(:UserColorType)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000002724a81, Ptr{Nothing} @0x00000000027b282b, Ptr{Nothing} @0x000000000ebc9de7, Ptr{Nothing} @0x00000000027b1afc, Ptr{Nothing} @0x00000000027b16f4, Ptr{Nothing} @0x00000000027b2048, Ptr{Nothing} @0x00000000027b2c85, Base.InterpreterIP in top-level CodeInfo for Plots at statement 3, Ptr{Nothing} @0x00000000027cf287, Ptr{Nothing} @0x00000000027cfdc1, Ptr{Nothing} @0x00000000027d0d0f, Ptr{Nothing} @0x000000000f1a4dfb, Ptr{Nothing} @0x000000000f1a4a1f, Ptr{Nothing} @0x

In [2]:
@with_kw struct RacecarParameters
    # Rewards
    r_crash::Real = -100 #Terminal State
    r_avoided_stop::Real = -4 #Terminal State
    r_avoided_continue::Real = 20 # No Crash yet by continuing straight.
    r_avoided_left::Real = -5 #Terminal State
    r_avoided_right::Real = -6 #Terminal State
    
    # Transition probability
    p_threat_arises::Real = .15   #Probability that the car behind is going to be a threat
    p_crash_when_continue_wthreat::Real = .5
    #Obervation probabilities
    p_collision_detected::Real = .7 # Suboptimal conditions when the lidar only has a 70% chance of detecting the collision
    p_false_alarm::Real = .1        # when no threat
end

RacecarParameters

In [3]:
params = RacecarParameters();

In [4]:
begin
    @enum State Crash Collision_AC Collision_AS Collision_AL Collision_AR 
    @enum Action Continue Stop Left_T Right_T 
    @enum Observation Collision_D Collision_U
end

In [5]:
𝒮 = [Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR]

5-element Vector{State}:
 Crash::State = 0
 Collision_AC::State = 1
 Collision_AS::State = 2
 Collision_AL::State = 3
 Collision_AR::State = 4

In [6]:
𝒜 = [Continue, Stop, Left_T, Right_T]

4-element Vector{Action}:
 Continue::Action = 0
 Stop::Action = 1
 Left_T::Action = 2
 Right_T::Action = 3

In [7]:
𝒪 = [Collision_D, Collision_U]

2-element Vector{Observation}:
 Collision_D::Observation = 0
 Collision_U::Observation = 1

In [8]:
initialstate_distr = Deterministic(Collision_AC)

Deterministic{State}(Collision_AC)

In [9]:
function T(s::State, a::Action)
    p_Crash_I::Real = params.p_crash_when_continue_wthreat
    p_Threat::Real = params.p_threat_arises
    p_Crash::Real = p_Threat*p_Crash_I
    if a == Right_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [.1,0,0,0,.9])
    elseif a == Left_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [.1,0,0,.9,0]) 
    elseif a == Continue && s == Collision_AC
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [p_Crash,1-p_Crash,0,0])
    elseif s == Crash
         return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [1,0,0,0,0])
   elseif a == Stop || s == Collision_AS
         return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,100,0,0])
    else
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [p_Crash,1-p_Crash,0,0])
    end
end

T (generic function with 1 method)

In [10]:
function O(s::State, a::Action, sp::State)
    p_Threat::Real = params.p_threat_arises
    p_True_Detect = params.p_collision_detected
    p_False_Detect = params.p_false_alarm
    p_Normal = p_Threat*p_True_Detect + (1-p_Threat)*p_False_Detect
    if sp == Crash
        return SparseCat([Collision_D, Collision_U], [p_True_Detect, 1-p_True_Detect])
    elseif sp == Collision_AC
        return SparseCat([Collision_D, Collision_U], [p_False_Detect, 1-p_False_Detect])
    else
        return SparseCat([Collision_D, Collision_U], [p_Normal,1-p_Normal])
    end
end        

O (generic function with 1 method)

In [11]:
function R(s::State, a::Action)
    return(s == Crash ? params.r_crash : 0) + (a == Continue ? params.r_avoided_continue : 0) + 
                                              (a == Stop ? params.r_avoided_stop : 0) +
                                              (a == Left_T ? params.r_avoided_left : 0) + 
                                              (a == Right_T ? params.r_avoided_right : 0) 
end

R (generic function with 1 method)

In [12]:
# R(Collision_AC, Continue)

In [13]:
γ = 0.9

0.9

In [14]:
abstract type Racecar <: POMDP{State, Action, Observation}end

In [15]:
pomdp = QuickPOMDP(Racecar,
    states = 𝒮,
    actions = 𝒜,
    observations = 𝒪,
    transition = T,
    reward = R,
    observation = O,
    discount = γ,
    initialstate = initialstate_distr);

In [16]:
struct ActWhenThreat <: Policy end

In [17]:
struct ActWhenBelievedThreat <: Policy end

In [18]:
const Belief = Vector{Real};

In [19]:
function POMDPs.action(::ActWhenThreat, o::Observation)
    return o == Collision_D ? Stop : Continue
end;

In [20]:
function POMDPs.action(::ActWhenBelievedThreat, b::Belief)
    if b[1] > b[2]
        return Stop
    else
        return Continue
    end
end;

In [21]:
updater(pomdp::QuickPOMDP{Racecar}) = DiscreteUpdater(pomdp)

updater (generic function with 1 method)

In [22]:
b0 = uniform_belief(pomdp); b0.b

5-element Vector{Float64}:
 0.2
 0.2
 0.2
 0.2
 0.2

In [23]:
begin 
    a1 = Continue 
    o1 = Collision_U
    b1 = update(DiscreteUpdater(pomdp), b0, a1, o1)
    b1.b
end

5-element Vector{Float64}:
 0.004382042568413523
 0.02978000357717761
 0.9658379538544088
 0.0
 0.0

In [24]:
using BasicPOMCP

In [25]:
pomcp_solver = POMCPSolver()

POMCPSolver
  max_depth: Int64 20
  c: Float64 1.0
  tree_queries: Int64 1000
  max_time: Float64 Inf
  tree_in_info: Bool false
  default_action: ExceptionRethrow ExceptionRethrow()
  rng: Random._GLOBAL_RNG Random._GLOBAL_RNG()
  estimate_value: RolloutEstimator


In [26]:
pomcp_planner = solve(pomcp_solver, pomdp);

In [27]:
initialstate(pomdp)

Deterministic{State}(Collision_AC)

In [28]:
aₚ, info = action_info(pomcp_planner, initialstate(pomdp), tree_in_info=true); aₚ

Continue::Action = 0

In [29]:
using D3Trees

In [30]:
pomcp_tree = D3Tree(info[:tree], init_expand=3)

o: <root>
N: 1000      
├──a: Continue
│  N: 957    
│  V: -12.5     
│  ├──o: Collision_D
│  │  N: 132       
│  │  ├──a: Continue
│  │  │  N: 1      
│  │  │  V: -337       (1 children)
│  │  ├──a: Stop
│  │  │  N: 1      
│  │  │  V: -256       (1 children)
│  │  ├──a: Left_T
│  │  │  N: 1      
│  │  │  V: -167       (1 children)
│  │  └──a: Right_T
│  │     N: 129    
│  │     V: -111       (2 children)
│  └──o: Collision_U
│     N: 823       
│     ├──a: Continue
│     │  N: 814    
│     │  V: -22.4      (2 children)
│     ├──a: Stop
│     │  N: 1      
│     │  V: -358       (1 children)
│     ├──a: Left_T
│     │  N: 7      
│     │  V: -75.3      (1 children)
│     └──a: Right_T
│        N: 1      
│        V: -89.4      (1 children)
├──a: Stop
│  N: 1      
│  V: -88.8     
│  └──o: Collision_D
│     N: 0         
│     ├──a: Continue
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Stop
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Left_T
│     │  N: 0      
│     │  V: 0          (0 children)
│     └──a: Right_T
│        N: 0      
│        V: 0          (0 children)
├──a: Left_T
│  N: 29     
│  V: -88       
│  ├──o: Collision_D
│  │  N: 5         
│  │  ├──a: Continue
│  │  │  N: 1      
│  │  │  V: -313       (1 children)
│  │  ├──a: Stop
│  │  │  N: 1      
│  │  │  V: -120       (1 children)
│  │  ├──a: Left_T
│  │  │  N: 1      
│  │  │  V: -118       (1 children)
│  │  └──a: Right_T
│  │     N: 2      
│  │     V: -30.1      (1 children)
│  └──o: Collision_U
│     N: 22        
│     ├──a: Continue
│     │  N: 13     
│     │  V: -83.1      (2 children)
│     ├──a: Stop
│     │  N: 1      
│     │  V: -129       (1 children)
│     ├──a: Left_T
│     │  N: 7      
│     │  V: -92.7      (1 children)
│     └──a: Right_T
│        N: 1      
│        V: -172       (1 children)
└──a: Right_T
   N: 13     
   V: -93.4     
   ├──o: Collision_U
   │  N: 9         
   │  ├──a: Continue
   │  │  N: 3      
   │  │  V: -44.6      (1 children)
   │  ├──a: Stop
   │  │  N: 1      
   │  │  V: -15.6      (1 children)
   │  ├──a: Left_T
   │  │  N: 2      
   │  │  V: -124       (1 children)
   │  └──a: Right_T
   │     N: 3      
   │     V: -26.1      (1 children)
   └──o: Collision_D
      N: 2         
      ├──a: Continue
      │  N: 0      
      │  V: 0          (0 children)
      ├──a: Stop
      │  N: 1      
      │  V: -236       (1 children)
      ├──a: Left_T
      │  N: 0      
      │  V: 0          (0 children)
      └──a: Right_T
         N: 1      
         V: -402       (1 children)

In [31]:
inchrome(pomcp_tree)

Process(`cmd /C start chrome 'C:\Users\cveda\AppData\Local\Temp\jl_0j210V\tree.html'`, ProcessExited(0))

In [32]:
using POMCPOW

In [33]:
pomcpow_solver = POMCPOWSolver()

POMCPOWSolver{Random._GLOBAL_RNG}
  eps: Float64 0.01
  max_depth: Int64 9223372036854775807
  criterion: MaxUCB
  final_criterion: MaxQ MaxQ()
  tree_queries: Int64 1000
  max_time: Float64 Inf
  rng: Random._GLOBAL_RNG Random._GLOBAL_RNG()
  node_sr_belief_updater: POMCPOW.POWNodeFilter POMCPOW.POWNodeFilter()
  estimate_value: RolloutEstimator
  enable_action_pw: Bool true
  check_repeat_obs: Bool true
  check_repeat_act: Bool true
  tree_in_info: Bool false
  alpha_observation: Float64 0.5
  k_observation: Float64 10.0
  alpha_action: Float64 0.5
  k_action: Float64 10.0
  init_V: Float64 0.0
  init_N: Int64 0
  next_action: MCTS.RandomActionGenerator{Random._GLOBAL_RNG}
  default_action: ExceptionRethrow ExceptionRethrow()


In [34]:
pomcpow_planner = solve(pomcpow_solver, pomdp);

In [35]:
aₚ, info = action_info(pomcpow_planner, initialstate(pomdp), tree_in_info=true); aₚ

Continue::Action = 0

In [36]:
using D3Trees

In [37]:
pomcpow_tree = D3Tree(info[:tree], init_expand=3)

o: <root>
N: 1000      

├──a: Continue
│  N: 988     V: -32.9     
│  ├──o: Collision_U
│  │  N: 835       

│  │  ├──a: Continue
│  │  │  N: 1       V: -127       (1 children)
│  │  ├──a: Stop
│  │  │  N: 9       V: -110       (2 children)
│  │  ├──a: Left_T
│  │  │  N: 40      V: -109       (2 children)
│  │  └──a: Right_T
│  │     N: 785     V: -47.8      (2 children)
│  └──o: Collision_D
│     N: 151       

│     ├──a: Right_T
│     │  N: 3       V: -115       (1 children)
│     ├──a: Left_T
│     │  N: 2       V: -181       (1 children)
│     ├──a: Stop
│     │  N: 145     V: -95.9      (2 children)
│     └──a: Continue
│        N: 1       V: -372       (1 children)
├──a: Left_T
│  N: 10      V: -89.9     
│  ├──o: Collision_D
│  │  N: 2         

│  │  ├──a: Left_T
│  │  │  N: 1       V: -220       (1 children)
│  │  └──a: Stop
│  │     N: 1       V: -330       (1 children)
│  └──o: Collision_U
│     N: 6         

│     ├──a: Right_T
│     │  N: 1       V: -56.3      (1 children)
│     ├──a: Left_T
│     │  N: 2       V: -13.9      (1 children)
│     ├──a: Continue
│     │  N: 1       V: -177       (1 children)
│     └──a: Stop
│        N: 2       V: -55.1      (1 children)
├──a: Right_T
│  N: 1       V: -137      
│  └──o: Collision_U
│     N: 0         

└──a: Stop
   N: 1       V: -106      
   └──o: Collision_U
      N: 0

In [38]:
inchrome(pomcpow_tree)

Process(`cmd /C start chrome 'C:\Users\cveda\AppData\Local\Temp\jl_TWISYc\tree.html'`, ProcessExited(0))